In [118]:
import plotly.express as px
import allel
import numpy as np
import pandas as pd

def load_vcf(vcf_path, metadata):
    """
    Load VCF and filter poor-quality samples
    """
    
    sampleIDs = metadata.sampleID.to_list()
    
    # load vcf and get genotypes and positions
    vcf = allel.read_vcf(vcf_path, fields='*')
    samples = vcf['samples']
    # keep only samples in qcpass metadata 
    sample_mask = np.isin(vcf['samples'], metadata.sampleID)
    
    # remove low quality samples 
    geno = allel.GenotypeArray(vcf['calldata/GT'])
    geno = geno.compress(sample_mask, axis=1)
    pos = vcf['variants/POS']
    contig = vcf['variants/CHROM']
    indel = vcf['variants/INDEL']
    
    # remove indels 
    geno = geno.compress(~indel, axis=0)
    pos = pos[~indel]
    contig = contig[~indel]
    ref = vcf['variants/REF'][~indel]
    alt = vcf['variants/ALT'][~indel]
    ann = read_ANN_field(vcf_path)[~indel]
    
    return geno, pos, contig, samples[sample_mask], ref, alt, ann

def read_ANN_field(vcf_file):
    anns = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue  # Skip header lines
            fields = line.strip().split('\t')
            info_field = fields[7]
            info_pairs = info_field.split(';')
            ann_value = None
            for pair in info_pairs:
                if pair.startswith('ANN='):
                    ann_value = pair.split('=')[1]
                    break
            anns.append(ann_value)

    return np.array(anns)

def vcf_to_snp_dataframe(vcf_path, metadata):

    geno, pos, contig, samples, ref, alt, ann = load_vcf(vcf_path=vcf_path, metadata=metadata)
    
    # make dataframe of variant positions and merge with bed
    snp_df = pd.DataFrame({'contig':contig, 'pos':pos, 'ref':ref, 'alt':[list(a[a != ""]) for a in alt], 'ann':ann})
    snp_df = snp_df.merge(bed_df)

    # filter out AIMs (will only do anything if ag-vampir)
    aim_mask = snp_df.eval("~target_id.str.contains('AIM')")
    geno = geno.compress(aim_mask, axis=0)
    snp_df = snp_df[aim_mask].reset_index(drop=True)
    
    # split multiallelic rows but store an index of the variant and alternate allele
    snp_df = snp_df.explode('alt').reset_index().rename(columns={'index':'variant_index'})
    snp_df = snp_df.assign(alt_index=snp_df.groupby('target_id').cumcount()) 
    snp_df = snp_df.assign(label=lambda x: x.target_id + " | " +  x.alt.fillna('NA'))
    
    return snp_df, geno

def calculate_frequencies_cohort(snp_df, geno, cohort_col):
    
    df = snp_df.copy()
    
    # get indices of each cohort
    coh_dict = {}
    cohs = metadata[cohort_col].unique()
    cohs = cohs[~pd.isnull(cohs)]
    for coh in cohs:
        coh_dict[coh] = np.where(metadata[cohort_col] == coh)[0]

    # get allele counts for each population
    ac = geno.count_alleles_subpops(coh_dict)
    
    for coh in cohs:
        total_counts = []
        alt_counts = []
        for i, row in df.iterrows():
            var_idx = row['variant_index']
            alt_idx = row['alt_index']
            total_counts.append(ac[coh][var_idx,:].sum())
            alt_counts.append(ac[coh][var_idx, alt_idx])

        df.loc[:, f'count_{coh}'] = np.array(alt_counts)
        df.loc[:, f'frq_{coh}'] = np.round(np.array(alt_counts)/np.array(total_counts), 3)
    
    freq_df = df.set_index('label').filter(like='frq')
    freq_df.columns = freq_df.columns.str.replace("frq_", "")
    
    return freq_df

def plot_allele_frequencies(df, cohort_col):
        
    fig = px.imshow(
            img=df,
            zmin=0,
            zmax=1,
            width=np.max([400, df.shape[1] * 100]),
            height=df.shape[0] * 20,
            text_auto=True,
            aspect=1,
            color_continuous_scale="Reds",
            title=f"Allele frequencies | by {cohort_col}",
        )
    fig.update(layout_coloraxis_showscale=False)

    fig.show()

In [77]:
dataset = 'ampseq-vigg-01'
metadata_path = "../../results/config/metadata.qcpass.tsv"
cohort_cols = 'taxon,location'
bed_path = "../../config/ag-vampir.bed"
vcf_path = "../../results/vcfs/targets/ampseq-vigg-01.annot.vcf"

### Plotting allele frequencies

This page shows allele frequencies in each cohort of the SNPs genotyped in the amplicon sequencing protocol.

In [54]:
cohort_cols = cohort_cols.split(",")

bed_df = pd.read_csv(bed_path, sep="\t", header=None, names=['contig', 'start', 'pos', 'amplicon_id', 'target_id'])

# load metadata
if metadata_path.endswith('.xlsx'):
    metadata = pd.read_excel(metadata_path, engine='openpyxl')
elif metadata_path.endswith('.tsv'):
    metadata = pd.read_csv(metadata_path, sep="\t")
elif metadata_path.endswith('.csv'):
    metadata = pd.read_csv(metadata_path, sep=",")
else:
    raise ValueError("Metadata file must be .xlsx or .csv")

In [119]:
snp_df, geno = vcf_to_snp_dataframe(vcf_path, metadata)

/home/sanj/.local/lib/python3.10/site-packages/allel/io/vcf_read.py:1732: UserWarning:

invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'



In [126]:
df = snp_df.assign(ann=lambda x: x.ann.str.split(","))

In [127]:

anns = []
for i, row in df.iterrows():
    ann = snp_df

In [132]:
row['ann'][row['alt_index']]

'T|downstream_gene_variant|MODIFIER|AGAP000819|AGAP000819|transcript|AGAP000819-RA|protein_coding||c.*1153A>T|||||1153|'

In [123]:
snp_df['ann'][0].split(",")

['A|missense_variant|MODERATE|ACE1|AGAP001356|transcript|AGAP001356-RA|protein_coding|4/8|c.838G>A|p.Gly280Ser|1075/6063|838/2214|280/737||',
 'T|missense_variant|MODERATE|ACE1|AGAP001356|transcript|AGAP001356-RA|protein_coding|4/8|c.838G>T|p.Gly280Cys|1075/6063|838/2214|280/737||']

In [55]:
snp_df, geno = vcf_to_snp_dataframe(vcf_path, metadata)

for cohort_col in cohort_cols:
    
    freq_df = calculate_frequencies_cohort(
        snp_df=snp_df, 
        geno=geno, 
        cohort_col=cohort_col
    )

    plot_allele_frequencies(
        df=freq_df,
        cohort_col=cohort_col
    )

/home/sanj/.local/lib/python3.10/site-packages/allel/io/vcf_read.py:1732: UserWarning:

invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'

/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide



/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_121751/1310824900.py:77: RuntimeWarning:

invalid value encountered in divide

